##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Play with AI - 猜字遊戲

這本 Cookbook 說明了如何使用調整指令版本的 Gemma 模型作為聊天機器人來玩"猜字遊戲"。

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemma-cookbook/blob/zh-tw/Gemma/Guess_the_word.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 執行</a>
  </td>
</table>


## 設定

### 選擇 Colab 執行環境
要完成此指南，你需要有一個具有足夠資源的 Colab 執行環境來執行 Gemma 模型。在這種情況下，你可以使用 T4 GPU:

1. 在 Colab 視窗的右上角，選擇 **▾ (其他連接選項)** 。
2. 選擇 **變更執行環境類型** 。
3. 在 **硬體加速器** 下，選擇 **T4 GPU** 。

### 在 Kaggle 上設定 Gemma
要完成此指南，你首先需要完成 [Gemma 設定](https://ai.google.dev/gemma/docs/setup)中的設定指示。Gemma 設定指示會告訴你如何進行以下操作:

* 獲取在 kaggle.com 上使用 Gemma 的權限。
* 選擇一個具有足夠資源的 Colab 執行環境來執行 Gemma 2B 模型。
* 生成並配置 Kaggle 用戶名和 API 金鑰。

完成 Gemma 設定後，繼續下一部分，在那裡你將為你的 Colab 環境設置環境變數。


### 設定環境變數

設定環境變數為 `KAGGLE_USERNAME` 和 `KAGGLE_KEY`.


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### 安裝相依套件

安裝 Keras 和 KerasNLP.


In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatibl

### 建立一個聊天助手來管理對話狀態


In [ ]:
import re

import keras
import keras_nlp

# Run at half precision to fit in memory
keras.config.set_floatx("bfloat16")

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_2b_en")
gemma_lm.compile(sampler="top_k")


class GemmaChat:

    __START_TURN__ = "<start_of_turn>"
    __END_TURN__ = "<end_of_turn>"

    def __init__(self, model, system="", history=None):
        self.model = model
        self.system = system
        if not history:
            self.history = []
        else:
            self.history = history

    def format_message(self, message, prefix=""):
        return f"{self.__START_TURN__}{prefix}\n{message}{self.__END_TURN__}\n"

    def add_to_history(self, message, prefix=""):
        formated_message = self.format_message(message, prefix)
        self.history.append(formated_message)

    def get_full_prompt(self):
        prompt = "\n".join([self.system + "\n", *self.history])
        return prompt

    def send_message(self, message):
        self.add_to_history(message, "user")
        prompt = self.get_full_prompt()
        response = self.model.generate(prompt, max_length=4096)
        # print('--------------------response---------------------------')
        # print(response)
        # print('--------------------response---------------------------')
        result = response.replace(prompt, "")
        self.add_to_history(result, "model")
        return result

    def show_history(self):
        for h in self.history:
            print(h)


chat = GemmaChat(gemma_lm)

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metada

## 玩這個遊戲


In [ ]:
theme = input("Choose your theme: ")
setup_message = f"Generate a random single word from {theme}."

chat.history.clear()
answer = chat.send_message(setup_message).split()[0]
answer = re.sub(r"\W+", "", answer)  # excludes all numbers, letters and '_'
chat.history.clear()
cmd_exit = "quit"
question = f'Describe the word "{answer}" without saying it.'

resp = ""
while resp.lower() != answer.lower() and resp != cmd_exit:
    text = chat.send_message(question)
    if resp == "":
        print(f'Guess what I\'m thinking.\nType "{cmd_exit}" if you want to quit.')
    remove_answer = re.compile(re.escape(answer), re.IGNORECASE)
    text = remove_answer.sub("XXXX", text)
    print(text)
    resp = input("\n> ")

if resp == cmd_exit:
    print(f"The answer was {answer}.\n")
else:
    print("Correct!")

Choose your theme: animal
Guess what I'm thinking.
Type "quit" if you want to quit.
The word evokes powerful images in the minds of many, a symbol of strength, courage, and dominance.

> tiger
**Answer:** A creature with a powerful and majestic presence, known for its courage, strength, and fierce nature.

> lion
Correct!
